# 00 - Introduction

What is metaprogramming?

Wikipedia: A programming technique in which computer programs have the ability to treat other programs as their data.

It means that a program can be designed to read, generate, analyse, or transform other programs, and even modify itself while running.

The basic idea is we can use **code** to modify **code**.

Here are some examples:

- decorators: use a function to modify the behaviour of another function (typically).
- descriptors: use code to modify the behaviour of the dot (`.`) operator.
- metaclasses:
    - use code to create classes (types; same thing). Can be though of class (type) factory.
    - Superficially, metaclasses are not difficult to understand, but the details can get complicated.
    - Knowing when to use them is not easy. Generally speaking, they're useful for libraries/frameworks, not for application code.
    - This section will take lots of time to understand and absorb - re-reading will probably be necessary.

# 01 - Decorators and Descriptors - Review

Covered extensively in other summaries - no need to repeat here.

# 02 - The `__new__` Method

We've studied the `__init__` method quite a bit so far. It is basically a method that gets called right after the class instance has been created, usually invoked when we call the Class with arguments to instantiate an instance.

The `__new__` method is the method that is invoked to actually create the new object, as an instance of the desired class.

Since the `object` class provides a default implementation for `__new__` we rarely have to bother with it, but sometimes we want to intercept the instance creation to tweak things a bit.

The `__new__` method, unlike the `__init__` method is actually a **static** method, not an **instance** method. Which kinds of make sense since the instance does not exist yet - that's what the `__new__` method is trying to create.

Why it's not a **class** method is more complicated. We'll see why that's the case as we explore `__new__`.

Remember how we create instances of a class - we call the class with whatever arguments we need to initialize the class state:

```python
p = Person(name, age)
```

The creation of the class instance is then done in two steps:
1. The `__new__` method is called via inheritance from `object`. It receives, as arguments, the class object we want an instance of, and any additional arguments we pass to the creation call (e.g. `name` and `age`). It should return a new instance of the class (and it may have used the arguments to initialize stuff in the class too, that's up to how you write your `__new__` method)
3. If the object returned by `__new__` is an instance of the class specified in the call to `__new__`, then the `__init__` method is also called. The `__init__` method is an instance method and does not return anything (well, it returns None).

The manual approach to creating an instance of the class above is as follows:
```python
p = object.__new__(Person)
p.__init__(name, age)
```
There are some small differences between these two approaches which we'll cover soon.

The `__new__` method has the following parameters: `object.__new__(class, *args, **kwargs)`
- The `*args` and `**kwargs` are ignored, **but**, they must match what we intend to pass to the `__init__`of `class`
- A new object of type `class` is returned

##### Overriding the `__new__` method

We *can* override this method in our own custom classes. 

This should return a new object which is an instance of `class` but it doesn't *have to*, the consequence being that `__init__` is no longer called on the instance of `class`.

We don't typically override the internal implementations of `__new__`; rather, we do something before/after creating the object which we perform by delegating to `super().__new__` - similar to how we use decorators. 

We typically move all of the `__init__` logic into `__new__` after the instance has been instantiated because that's no different to running the logic in `__init__`.

So, **in metaprogramming, we should only implement one of the two**.

Here's how we can do it in practice:

In [6]:
class Person:
    def __new__(cls, name, age):
        # do stuff here
        instance = super().__new__(cls)  
        # do stuff here
        return instance

When we call `Person('Guido', 68)`:
- Python calls `__new__(Person, 'Guido', 68)`
- `__new__` returns an object
    - if that object is of the same type as the "requested":
        - `new_object.__init__('Guido', 68)` is called
    - else:
        - don't attempt to call the `__init__`.

Note: 
- We don't need to pass the other *args/**kwargs to the `super().__new__()`, just the `Person.__new__()`. If we define an `__init__` for this `Person` class, we must have `name` and `age` as parameters.
- Using `object.__new__` instead of `super().__new__` wil raise issues when dealing with inheritance; child classes will be instantiated and initialised but parent classes will only be initialised, **not instantiated**.

#### Code

When we want to inherit from the builtin types such as `int`, `float`, etc., we can only override the `__new__`, not the `__init__` due to the implementation details in the C language.

So, if we wanted to create an integer using the `int()` constructor, but instead of just returning the integer, we return the square of it, we may try: 

In [3]:
class Squared(int):
    def __init__(self, x):
        super().__init__(x ** 2)

res = Squared(4)

TypeError: object.__init__() takes exactly one argument (the instance to initialize)

But the correct approach is:

In [5]:
class Squared(int):
    def __new__(cls, x):
        return super().__new__(cls, x ** 2)

Squared(4)

16

Here's a quick example:

Let's say we want to write a bound method that calculates the area. One (unconventional) way we could do this is: 

In [17]:
class Square:
    area = lambda self: self.w * self.l

    def __init__(self, w, l):
        self.w = w
        self.l = l

s = Square(3, 4)

s.area()

12

`area` is an honest-to-goodness bound method of `s`, which means the first argument is the instance `self`.

In [25]:
s.area

<bound method Square.<lambda> of <__main__.Square object at 0x000001C0B8207640>>

Another way we can do this is via `__new__`:

In [27]:
class Square:
    def __new__(cls, w, l):
        cls.area = lambda self: self.w * self.l
        instance = super().__new__(cls)
        instance.w = w
        instance.l = l
        return instance

s = Square(3, 4)

s.area()

12

# 03 - How Classes are Created

Classes are instances of `type`, so we often say classes *are* types, and types are callable. Like all other objects in Python, `type` inherits from `object`.

#### Inner mechanics of class creation

Let's take the following code as an example:
```python
class Person:
    planet = 'Earth'
    name = property(fget=lambda self: self._name)

    def __init__(self, name):
    self._name = name
```

Here are the following steps of class creation:

1. The class **body** is extracted (and treated as a blob of text for now).
2. A new dictionary is created - this will be the namespace of the new class.
3. The body code is executed which populates the namespace, adding symbols (keys) for `planet`, `name`, `__init__`. (If this is confusing, just imagine this body of text in the global (module) namespace. The module namespace will have these exact symbols.)
4. Python uses the `type` class to create a new instance of `type` using the name of the class `Person`, the classes it inherits from, and the namespace made previously. -> `type(class_name, class_bases, class_dict)`.
5. Just as calling `Person('Alex')` returns an instance of `Person`, calling `type('Person', ...)` returns an instance of `type`.

To get an idea of how this works, we can visualise the process in code with the `exec(object, globals, locals)` function:

Passing `globals()` into `globals` lets the function see our global scope whereas `locals` is the dictionary to be used to store all the symbols for the executed program.

In [2]:
namespace = {}
exec('''
a = 20
b = 20
def __init__(self):
    pass
''', globals(), namespace)

In [3]:
namespace

{'a': 20, 'b': 20, '__init__': <function __main__.__init__(self)>}

As you can see, our namespace got populated with our symbols. This is exactly what happens with our class **body**

##### Manual Class Creation

The `type` function has two primary ways of being called:
- with a single argument -> returning the type of that argument
- with three arguments (name, bases, dict) -> returning a new type

The second approach is what we're exploring. 

In [11]:
class_name = 'Circle'

Let's deal with populating a namespace using the class body first.

In [12]:
class_body = """
def __init__(self, x, y, r):
    self.x = x
    self.y = y
    self.r = r

def area(self):
    return math.pi * self.r ** 2
"""

In [13]:
class_bases = ()  # defaults to object

In [14]:
class_dict = {}

In [15]:
exec(class_body, globals(), class_dict)

In [16]:
class_dict

{'__init__': <function __main__.__init__(self, x, y, r)>,
 'area': <function __main__.area(self)>}

Now let's construct an instance of `type` with name `'Circle'`:

In [17]:
Circle = type(class_name, class_bases, class_dict)

In [20]:
Circle.__dict__

mappingproxy({'__init__': <function __main__.__init__(self, x, y, r)>,
              'area': <function __main__.area(self)>,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Circle' objects>,
              '__weakref__': <attribute '__weakref__' of 'Circle' objects>,
              '__doc__': None})

As you can see the `Circle` namespace dict contains our functions `__init__` and `area`.

In [19]:
c = Circle(0, 0, 1)

In [20]:
c.x, c.y, c.r

(0, 0, 1)

In [21]:
c.area()

3.141592653589793

So as you can see, we use the `type` class to construct new types (classes), basically creating instances of `type`.

This is why we refer to `type` as a **metaclass**. It is a class used to construct classes.

# 04 - Inheriting from type

Since `type` inherits from object, it has `__new__` and `__init__`. Calling `type` will call `__new__`. So..

`type(name, bases, class_dict)`

is similar

`type.__new__(type, name, bases, class_dict)`

which returns an instance of `type`. Remember of course that the `__init__` will no longer be called - but remember **in metaprogramming, we only implement one of the two**

Since `type` is a class, it can be used as a **base class** for a custom class: `MyClass = class MyType(type)`

We can also override `__new__` where we tweak things, but **delegate** to `type` for actually creating the `type`, e.g. for creating the class `Circle`, *not the instance of `Circle`.*

Note that, despite `MyClass` being used for **creating classes**, it is also, in itself, a class. It's a class used for creating classes - a metaclass - a type.

#### Example

Recall that the first argument of `__new__` is `cls` which will refer to the class we want to create an instance of. Since, this instance will be a class used to create other classes, I will call it `mcls`, short for metaclass.

and the remaining arguments should match the arguments of the `__init__`. 

Previously, we had to rely on the `type(name, bases, dict)` for creating our class, but if we inherit from it, we can do stuff before/after the creation process which we **delegate** back to `type`. 

Since, we'll be inheriting from `type`, calling `super().__new__` will perform the delegation.

In [31]:
import math

class CustomType(type):
    def __new__(mcls, name, bases, class_dict):
        print('Customized type creation!')
        cls_obj = super().__new__(mcls, name, bases, class_dict)  # delegate to super (type in this case)
        cls_obj.circ = lambda self: 2 * math.pi * self.r  # basically attaching a function to the class
        return cls_obj

Previously, we created `Circle` using `type(name, bases, dict)`. Now, let's go through the same process to create our `Circle` class using `CustomType` instead of `type`.

In [32]:
class_body = """
def __init__(self, x, y, r):
    self.x = x
    self.y = y
    self.r = r

def area(self):
    return math.pi * self.r ** 2
"""

And we create our class dictionary by executing the above code in the context of that dictionary:

In [33]:
class_dict = {}
exec(class_body, globals(), class_dict)

Then we create the `Circle` class:

In [34]:
Circle = CustomType('Circle', (), class_dict)

Customized type creation!


Available to our class is everything in the namespace which was populated using the `class_body` string:

In [35]:
c = Circle(0, 0, 1)
c.area()

3.141592653589793

But, don't forget! We also injected a bound method called `circ()` for instances of any classes created using the metaclass `CustomType`.

We actually injected this after **delegation**, but we could just have easily added it to our `class_dict` before delegation:

`class_dict['circ'] = lambda self: 2 * math.pi * self.r`

In [36]:
c.circ()

6.283185307179586

# 05 - Metaclasses

A **metaclass** of a class is defined as the class used to create that class.

By default, Python uses the `type` metaclass for creating our classes, but this can be overrided very easily, for example: `class Person(metaclass=MyType)`.

So, by default, Python does for example: `class Person(metaclass=type)`.

Putting it all together:

In [41]:
class MyType(type):
    def __new__(mcls, name, bases, cls_dict):
        print(f"Using custom metaclass {mcls.__name__} to create class {name}")
        # tweak things
        # delegate to `type` for class creation
        new_class = super().__new__(mcls, name, bases, cls_dict)
        # tweak some more
        # recall that __new__ should return the instance of what it promised to create
        return new_class

class Person(metaclass=MyType):
    def __init__(self, name):
        self.name = name

Using custom metaclass MyType to create class Person


This is identical to several tedious steps carried out in the previous section except Python takes care of all of this heavy lifting.

Python calls `MyType` with `MyType` itself as `mcls`, `Person` as the `name`, `object` as the `bases` (by default) and the `Person` class body as the `cls_dict`.

# 06 - Class Decorators

While metaclasses are very powerful, they can be hard to understand when reading code. Most of the time decorators work just as well while being much easier to read.

Writing class decorators is similar to writing function decorators except that the decorator needs to expect a class. Even parametrised decorators (decorator factory) are fairly intuitive: 

In [49]:
def apr(rate):
    def inner(cls):
        cls.apr = rate
        return cls
    return inner

@apr(0.02)
class SavingsAccount():
    pass

s = SavingsAccount()
s.apr

0.02

This is equivalent to:

In [50]:
class SavingsAccount():
    pass

SavingsAccount = apr(0.02)(SavingsAccount)

s = SavingsAccount()
s.apr

0.02

A metaclass can be used but we currently don't know how to pass the `rate` parameter via a metaclass approach and we'll also have inheritance issues.

So for this purpose, metaclasses are overkill! 

#### Example 1: Logger for all methods of a Class

Let's say we have a class with a number of methods. Each method takes a number of args and returns something. What if we wanted to log each call with their result?

We could decorate each method with a logger but that can become repetitive. Another option is to view the `vars` of the class and decorate anything that is a callable.

In [38]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        result = fn(*args, **kwargs)
        print(f'log: {fn.__qualname__}({args}, {kwargs}) = {result}')
        return result
    return inner

In [39]:
def class_logger(cls):
    for method_name, obj in vars(cls).items():
        if callable(obj):
            print("decorating: ", cls, obj)
            setattr(cls, method_name, func_logger(obj))
    return cls

So now we could do this:

In [40]:
@class_logger
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def greet(self):
        return f'Hello, my name is {self.name} and I am {self.age}'

decorating:  <class '__main__.Person'> <function Person.__init__ at 0x0000018858A6C3A0>
decorating:  <class '__main__.Person'> <function Person.greet at 0x0000018858A6FB50>


In [41]:
p = Person('John', 78)

log: Person.__init__((<__main__.Person object at 0x00000188576D3790>, 'John', 78), {}) = None


In [42]:
p.greet()

log: Person.greet((<__main__.Person object at 0x00000188576D3790>,), {}) = Hello, my name is John and I am 78


'Hello, my name is John and I am 78'

We will run into issues with static methods and class methods as they are **not** callables. (From Python 3.10, they are callables as they implement `__call__`, in addition to `__get__`.)


They are instead **descriptors** which are classes. The consequence of this is that if we want to decorate a method with `@staticmethod` or `@classmethod` in addition to a function decorator e.g. `@func_logger`, **we must apply the `@func_logger` first and __then__ the `@staticmethod`/`@classmethod`**

If however, we want to decorate our class as before but have it figure out how to deal with static and class methods, we can do that too.

Remember that all methods are **non-data descriptors** which means:
- They have the `__get__` method.
- Depending on how `__get__` is called, it either returns the function itself or the bound method. This is how python differentiates between the two.

We know that `__get__` takes 3 parameters. Here's what they are for regular functions:
- `self`: the function itself e.g. `add` because we have `add.__get__()`.
- `instance`: either `None` or our `__main__` module; we can think of our `__main__` module as the class housing this function, so calling it directly is like calling it from the module 'class' instead of an instance.
- `owner_class`: in this case, it will be our `__main__`module

If `instance=None`, `__get__` returns the function as opposed to a bound method. Where is the pointer to that function? In the `__func__` attribute of the descriptor.

In [43]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
            
        elif isinstance(obj, staticmethod):
            original_func = obj.__func__
            print('decorating static method', original_func)
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            print(method, type(method))
            setattr(cls, name, method)
            
        elif isinstance(obj, classmethod):
            original_func = obj.__func__
            print('decorating class method', original_func)
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
            
    return cls

In [44]:
@class_logger
class Person:
    @staticmethod
    def static_method(a, b):
        print('static_method called...', a, b)
        
    @classmethod
    def class_method(cls, a, b):
        print('class_method called...', cls, a, b)
        
    def instance_method(self, a, b):
        print('instance_method called...', self, a, b)

decorating: <class '__main__.Person'> static_method
decorating class method <function Person.class_method at 0x0000018858A6E560>
decorating: <class '__main__.Person'> instance_method


In [45]:
Person.static_method(10, 20)

static_method called... 10 20
log: Person.static_method((10, 20), {}) = None


In [46]:
Person.class_method(10, 20)

class_method called... <class '__main__.Person'> 10 20
log: Person.class_method((<class '__main__.Person'>, 10, 20), {}) = None


In [47]:
Person().instance_method(10, 20)

instance_method called... <__main__.Person object at 0x00000188576D6A10> 10 20
log: Person.instance_method((<__main__.Person object at 0x00000188576D6A10>, 10, 20), {}) = None


This is good, but what about **properties**?

As you may remember, the property class has an `fset`, `fget` and `fdel` attribute which are handles to their respective functions. The issue is that these are all read-only attributes.

But, remember that properties also have `getter()`, `setter()` and `deleter()` methods which take a function, stores it in the descriptor and then returns the descriptor itself. (Remember, this is why we can only call `@<some_prop>.setter` after applying the `@property` decorator. 

In [48]:
def class_logger(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
            
        elif isinstance(obj, staticmethod):
            original_func = obj.__func__
            print('decorating static method', original_func)
            decorated_func = func_logger(original_func)
            method = staticmethod(decorated_func)
            print(method, type(method))
            setattr(cls, name, method)
            
        elif isinstance(obj, classmethod):
            original_func = obj.__func__
            print('decorating class method', original_func)
            decorated_func = func_logger(original_func)
            method = classmethod(decorated_func)
            setattr(cls, name, method)
            
        elif isinstance(obj, property):
            print('decorating property', obj)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
    return cls

So, here we've checked to see if the property which is a descriptor has received `fget`. If it has, we want to replace the entire property with a new property with a modified `fget`. This modification involves taking the original `fget` function from our property, decorate it with `func_logger` and essentially creating a new entire property whose `fget` is equal to our decorated function.

Again, recall that the `getter()` of a property descriptor takes a function, stores it in the `fget` attribute of the descriptor, and returns a descriptor new descriptor. When we call the propery e.g. `Person.age`, the `__get__` is called on the `age` descriptor which calls the function stored in `fget`. 

In [49]:
class A:
    @property
    def x():
        print('original x called')

print(A.x)
A.x.fget()

original x called


In [50]:
def new_x():
    print('new x called')

A.x = A.x.getter(new_x)
print(A.x)
A.x.fget()

new x called


Let's now demonstrate it:

In [52]:
@class_logger
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name

decorating: <class '__main__.Person'> __init__
decorating property <property object at 0x0000018858AACF90>


In [53]:
p = Person('David')

log: Person.__init__((<__main__.Person object at 0x00000188576F8F10>, 'David'), {}) = None


In [54]:
p.name

log: Person.name((<__main__.Person object at 0x00000188576F8F10>,), {}) = David


'David'

Perfect!

But there's a problem...

We try to catch all functions in our class using the `callable` predicate function, but as we'll now show, **not all callables are functions**.

In [55]:
@class_logger
class Person:
    class Other:
        def __call__(self):
            print('called instance of Other...')
            
    other = Other()

decorating: <class '__main__.Person'> Other
decorating: <class '__main__.Person'> other


So, as you see it decorated both the class `Other` (since classes are callables), and it decorated `other` since we made instances of `Other` callable too.

So maybe we can use the `inspect` module to restrict our callables further.

In the original notebook, you can find the code to build up the following table that outlines how each predicate function in the `inspect` module treats static methods, class methods, instance methods, properties (`name`), dunder methods, classes within classes and instances of those (which have implemented `__call__`).

```
                   static_method	cls_method   	inst_method  	name         	__add__      	Other        	other        
isroutine          True         	True         	True         	False        	True         	False        	False        
ismethod           False        	False        	False        	False        	False        	False        	False        
isfunction         False        	False        	True         	False        	True         	False        	False        
isbuiltin          False        	False        	False        	False        	False        	False        	False        
ismethoddescriptor True         	True         	False        	False        	False        	False        	False   

The TLDR is that we want to use `isroutine` so that we can also decorate dunder methods, but we will still need our current implementation for properties, static methods and class methods. With some additional refactoring, we get:

In [57]:
import inspect

def class_logger(cls):
    for name, obj in vars(cls).items():
        if isinstance(obj, (staticmethod, classmethod)):
            type_ = type(obj)
            original_func = obj.__func__
            print(f'decorating {type_.__name__} method', original_func)
            decorated_func = func_logger(original_func)
            method = type_(decorated_func)
            setattr(cls, name, method)
            
        elif isinstance(obj, property):
            print('decorating property', obj)
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls, name, obj)
            
        elif inspect.isroutine(obj):
            print('decorating:', cls, name)
            setattr(cls, name, func_logger(obj))
    return cls

In [58]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        print('static_method called...')
    
    @classmethod
    def cls_method(cls):
        print('class method called...')
    
    def inst_method(self):
        print('instance method called...')
    
    @property
    def name(self):
        print('name getter called...')
    
    def __add__(self, other):
        print('__add__ called...')
    
    @class_logger
    class Other:
        def __call__(self):
            print(f'{self}.__call__ called...')
        
    other = Other()

decorating: <class '__main__.MyClass.Other'> __call__
decorating staticmethod method <function MyClass.static_method at 0x0000018858B08820>
decorating classmethod method <function MyClass.cls_method at 0x0000018858B088B0>
decorating: <class '__main__.MyClass'> inst_method
decorating property <property object at 0x0000018858ABEFC0>
decorating: <class '__main__.MyClass'> __add__


# 07 - Decorator Classes

First off, don't confuse this with class decorators - here I'm talking about using a class to create decorators - that can be used to decorate functions, or classes - but instead of the decorator being a function, it is a class whose instances will act as decorators.

We do this by making instances of the decorator class **callable**, by implementing the `__call__` method.

Here's a basic example to illustrate it:

In [1]:
class Logger:
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        print(f'Log: {self.fn.__name__} called.')
        return self.fn(*args, **kwargs)

In [2]:
@Logger
def say_hello():
    pass

In [3]:
say_hello()

Log: say_hello called.


**One limitation** with this approach is that we cannot decorate bound methods. Let's try it to see why we can't.

In [4]:
class Person:
    def __init__(self, name):
        self.name = name
        
    @Logger
    def say_hello(self):
        return f'{self.name} says hello!'

In [5]:
p = Person('David')

In [6]:
p.say_hello()

Log: say_hello called.


TypeError: Person.say_hello() missing 1 required positional argument: 'self'

We must first remember the difference between functions and methods.

Functions implement the **(non-data) descriptor protocol**: Implement the `__get__` method. If called from an instance, return a bound method. Otherwise, return the descriptor itself.

In [9]:
def my_func():
    pass
    
print(my_func)
hasattr(my_func, '__get__')

<function my_func at 0x000001C6C7D39F30>


True

In [11]:
class A:
    def my_func(self):
        pass

a = A()

print(A.my_func)  # returns the descriptor
print(a.my_func)  # returns a bound method using types.MethodType
hasattr(A.my_func, '__get__')

<function A.my_func at 0x000001C6C7F6D3F0>
<bound method A.my_func of <__main__.A object at 0x000001C6C908B940>>


True

Bound methods are created with `types.MethodType`

Here's a trimmed snippet on `MethodType` from `help`:
```
class method(object)
 |  method(function, instance)
 |  
 |  Create a bound instance method object.
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __func__
 |      the function (or other callable) implementing a method
 |  
 |  __self__
 |      the instance to which a method is bound
```

When creating our `MethodType`, the `function` parameter is the callable that will call our decorated function; in other words, it's our descriptor instance itself: `self`.

The `instance` is the object that the descriptor is bound to; in other words, it's the instance of the class containing the decorated method.

Let's create our modified `Logger`:

In [16]:
from types import MethodType

class Logger:
    def __init__(self, fn):
        self.fn = fn
        
    def __call__(self, *args, **kwargs):
        print(f'Log: {self.fn.__name__} called.')
        return self.fn(*args, **kwargs)
    
    def __get__(self, instance, owner_class):
        print(f'__get__ called: self={self}, instance={instance}')
        if instance is None:
            return self
        else:
            # self is callable, since it implements __call__
            return MethodType(self, instance)

In [17]:
class Person:
    def __init__(self, name):
        self.name = name
        
    @Logger
    def say_hello(self):
        return f'{self.name} says hello!'

In [18]:
p = Person('David')

In [19]:
p.say_hello

__get__ called: self=<__main__.Logger object at 0x000001C6C909D3C0>, instance=<__main__.Person object at 0x000001C6C905C9D0>


<bound method ? of <__main__.Person object at 0x000001C6C905C9D0>>

As you can see `say_hello` is now considered a bound method. And it bound the callable instance of Logger to the Person instance.

In [20]:
p.say_hello()

__get__ called: self=<__main__.Logger object at 0x000001C6C909D3C0>, instance=<__main__.Person object at 0x000001C6C905C9D0>
Log: say_hello called.


'David says hello!'

The last thing we should check is that the decorator works with class and static methods too.

Just remember that the order of the decorators is important - we need to decorate with our logger before we decorate with the static and class decorators. that way we end up decorating the decorated function (so just a plain fuinction decorator), and then making it into a class or static method.

In [23]:
class Person:
    @classmethod
    @Logger
    def cls_method(cls):
        print('class method called...')
        
    @staticmethod
    @Logger
    def static_method():
        print('static method called...')
        

In [24]:
Person.cls_method()

Log: cls_method called.
class method called...


In [25]:
Person.static_method()

Log: static_method called.
static method called...


# 08 - Metaclasses vs Class Decorators

The main advantages of class decorators over metaclasses are that you can:
- stack decorators,
- decorate classes in an inheritance chain with different decorators.

The main advantage of metaclasses over decorators is that:
- subclasses inherit parent metaclasses. In our example before with the class decorator `@class_logger`, any subclass of `Person`, such as `Student(Person)`, will not be logged. We would have to remember to decorate `Student` with `@class_logger`. We wouldn't have to do this if we used a metaclass instead.

# 09 - Metaclass Parameters

So this is the simplest form of creating metaclasses:

In [8]:
class MetaClass(type):
    def __new__(mcls, name, bases, class_dict):
        return super().__new__(mcls, name, bases, class_dict)

class MyClass(metaclass=MetaClass):
    pass

Can we add additional arguments to our `__new__`?

Yes, from Python 3.6 onwards:

In [9]:
class MetaClass(type):
    def __new__(mcls, name, bases, class_dict, arg1, arg2, arg3=None):
        return super().__new__(mcls, name, bases, class_dict)

class MyClass(metaclass=MetaClass, arg1=1, arg2=2, arg3=3):
    pass

Notice that the arguments **must** be passed as **named** arguments, except for default arguments like `arg3`. The reason why is because positional arguments are reserved for the parent objects that we inherit from.

Why might we want to pass additional arguments?

If we want to create a class with a number of class attributes that will only be determined at run time, we can pass these attributes in to the class definition. For example:

In [21]:
class AutoClassAttrib(type):
    def __new__(mcls, name, bases, cls_dict, extra_attrs=None):
        if extra_attrs:
            print('Creating class with some extra attributes: ', extra_attrs)
            # here I'm going to things directly into the cls_dict namespace
            # but could also create the class first, then add using setattr
            for attr_name, attr_value in extra_attrs:
                cls_dict[attr_name] = attr_value
        return super().__new__(mcls, name, bases, cls_dict)

In [22]:
class Account(metaclass=AutoClassAttrib, extra_attrs=[('account_type', 'Savings'), ('apr', 0.5)]):
    pass

Creating class with some extra attributes:  [('account_type', 'Savings'), ('apr', 0.5)]


In [23]:
Account.account_type, Account.apr

('Savings', 0.5)

An ever so slightly simpler way is the following:

In [35]:
class AutoClassAttrib(type):
    def __new__(mcls, name, bases, cls_dict, **extra_attrs):
        print('Creating class with some extra attributes: ', extra_attrs)
        cls_dict.update(extra_attrs)
        return super().__new__(mcls, name, bases, cls_dict)    

In [36]:
class Account(metaclass=AutoClassAttrib, account_type='Savings', apr=0.5):
    pass

Creating class with some extra attributes:  {'account_type': 'Savings', 'apr': 0.5}


In [38]:
Account.account_type, Account.apr

('Savings', 0.5)

# 10 - The `__prepare__` Method

We know that when we call `class MyClass(metaclass=MyMeta)`, the `__new__` of `MyMeta` is called with `__new__(mcls, name, bases, cls_dict)`.

We know `cls_dict` will become the namespace of our created classes, but where and how is the `cls_dict` populated?

In [17]:
class MyMeta(type):
    def __new__(mcls, name, bases, cls_dict):
        print(f"Using custom metaclass {mcls.__name__} to create class {name}")
        print(f"{cls_dict = }")
        new_class = super().__new__(mcls, name, bases, cls_dict)
        return new_class

In [18]:
class Person(metaclass=MyMeta):
    def __init__(self, name):
        self.name = name

Using custom metaclass MyMeta to create class Person
cls_dict = {'__module__': '__main__', '__qualname__': 'Person', '__init__': <function Person.__init__ at 0x0000026B46E42440>}


In fact, `__prepare__`, another static method implemented by `type`, is called before `__new__`.

`__prepare__` also receives any additional named args passed to our metaclass (`class MyClass(metaclass=MyMeta, <kwargs>)`) and must return a **mapping type**, e.g. an **empty** `dict`.

Python then manipulates the empty dict, injecting in what we see above e.g. `__module__`, `__qualname__`, etc. before calling `__new__` with this dictionary as `cls_dict`.

Let's replicate what Python does, but instead of returning an empty dictionary, we'll add some items. This will also demonstrate that Python is injecting what I mentioned above, not `__prepare__`.

In [22]:
class MyMeta(type):
    @staticmethod
    def __prepare__(name, bases, **kwargs):
        print('__prepare__ called...')
        print('\tname:', name)
        print('\tkwargs:', kwargs)
        return {'a': 100, 'b': 200}

    def __new__(mcls, name, bases, cls_dict, **kwargs):
        print('__new__ called...')
        print('\tcls_dict:', cls_dict)
        print('\tkwargs:', kwargs)
        return super().__new__(mcls, name, bases, cls_dict)

In [23]:
class MyClass(metaclass=MyMeta):
    pass

__prepare__ called...
	name: MyClass
	kwargs: {}
__new__ called...
	cls_dict: {'a': 100, 'b': 200, '__module__': '__main__', '__qualname__': 'MyClass'}
	kwargs: {}


Here's an easier way of seeing what `__prepare__` returns:

In [24]:
type.__prepare__()

{}

This gives us an alternative approach to populating our `cls_dict` in `__new__`. Previously we would do:

In [25]:
class MyMeta(type):
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        cls_dict.update(kwargs)
        return super().__new__(mcls, name, bases, cls_dict)

class MyClass(metaclass=MyMeta, arg1=100, arg2=200):
    pass

vars(MyClass)

mappingproxy({'__module__': '__main__',
              'arg1': 100,
              'arg2': 200,
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

But now we can do the following:

In [28]:
class MyMeta(type):
    def __prepare__(name, bases, **kwargs):
        return kwargs
        
    def __new__(mcls, name, bases, cls_dict, **kwargs):
        return super().__new__(mcls, name, bases, cls_dict)

class MyClass(metaclass=MyMeta, arg1=100, arg2=200):
    pass

vars(MyClass)

mappingproxy({'arg1': 100,
              'arg2': 200,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'MyClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyClass' objects>,
              '__doc__': None})

while ensuring `__new__` is defined because this lets us specify additional `kwargs`. By default, `__new__` does not accept additional arguments. 

As you can see above, the additional kwargs `arg1=100, arg2=200` were passed to `**kwargs` in `__prepare__`. 

Then Python injected some additional variables like `__module__`.

Then Python passed this dictionary to `cls_dict` in `__new__`.

#### Classes, Metaclasses, and `__call__`

Consider the following:

In [29]:
class Person:
    def __call__(self):
        pass

We know that this makes **instances** of this class a callable, not the class itself:

In [31]:
p = Person()  # not this
p()  # but this

But, since `Person` is also callable, it must implement `__call__`. Therefore, its metaclass, `type`, implements `__call__`.

Let's chronologically lay out the steps taken when `p = Person()` is called.

1. Firstly, `type.__call__` is called.
2. `type.__call__` then calls `Person.__new__` to initialise the class, returning an instance of `Person`.
3. `type.__call__` then calls `__init__` on the instance to instantiate it. This is then associated to the symbol `p`.

Okay, but we can also do:
```python
type(name, bases, cls_dict)
```

Therefore `type` is *also* a callable, which means *its* metaclass must implement `__call__`. 

What is the metaclass of `type`? Itself.

**The type of `type` is `type`.**

So what are the chronological steps when `type` is called - which occurs when `Person = type(name, bases, cls_dict)` is called, for example.

Pretty much the exact same steps as above.

1. Firstly, `type.__call__` is called.
2. `type.__call__` then calls `type.__new__` to initialise the class, returning an instance of `type` which is the `Person` **class**.
3. `type.__call__` then calls `__init__` on the instance (`Person` is the instance) -> `Person.__init__(). This is then associated to the symbol `Person`.

Let's compare and summarise by taking a look at the chronology of traditional **declarative** class creation.

When we type `class Person(metaclass=type)`:

1. `type.__prepare__` is called returning an empty dictionary.
2. An internal python process runs to populate the dictionary with `__qualname__`, `__docs__`, etc, prepares the class body, the bases, etc., before finally calling `type.__call__`.
3. `type.__call__` calls `type.__new__` to initialise the class, returning an instance of `type` which is the `Person` **class**.
4. `type.__call__` then calls `__init__` on the instance (`Person` is the instance) -> `Person.__init__()`. This is then associated to the symbol `Person`.

# 11 - Metaprogramming Application 1

Are you tired of writing boiler-plate code like this:

In [1]:
class Point2D:
    __slots__ = ('_x', '_y')
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
        
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    def __eq__(self, other):
        return isinstance(other, Point2D) and (self.x, self.y) == (other.x, other.y)
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __repr__(self):
        return f'Point2D({self.x}, {self.y})'
    
    def __str__(self):
        return f'({self.x}, {self.y})'
        
class Point3D:
    __slots__ = ('_x', '_y', '_z')
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z
    
    @property
    def x(self):
        return self._x
    
    @property
    def y(self):
        return self._y
    
    @property
    def z(self):
        return self._z
    
    def __eq__(self, other):
        return isinstance(other, Point3D) and (self.x, self.y, self.z) == (other.x, other.y, other.z)
    
    def __hash__(self):
        return hash((self.x, self.y, self.z))

    def __repr__(self):
        return f'Point2D({self.x}, {self.y}, {self.z})'
    
    def __str__(self):
        return f'({self.x}, {self.y}, {self.z})'


It's basically the opposite of DRY!

Let's try to solve this problem using metaclasses (instead of decorators because we might care about inheritance).

We want to create a base class of sorts that has implemented all of our slots, properties and dunder methods. In the end, we want to have something that looks like:
```python
class Person(metaclass=SlottedStruct):
    _fields = ['name', 'age']
    
    def __init__(self, name, age):
        self._name = name
        self._age = age
```
where all slots, properties and dunder methods are defined. 

Our approach will be to find the fields of the class in the `_fields` class attribute, and use this to create appropriately named slots and properties. 

We will also create all of our methods and then inject them into our class object.

In [15]:
class SlottedStruct(type):
    def __new__(mcls, name, bases, cls_dict):
        cls_object = super().__new__(mcls, name, bases, cls_dict)
        
        # setup the __slots__
        cls_object.__slots__ = [f'_{field}' for field in cls_object._fields]

        # setup the properties
        for field in cls_object._fields:
            # this is what we want to write:
            # setattr(cls_obj, field, property(fget=lambda self: getattr(self, f'_{field}')
            # but we can't as `property(...)` is a closure and
            # `field` is a free variable because it's outside the closure's scope.
            # Therefore, by the end of this for loop, all lambdas produced will point to the same `field` variable.
            # We need to set it in stone now, and we do that by assigning it as a default value in the lambda.
            # Remember that default values are computed at compile time, not runtime.
            setattr(cls_object, field, property(fget=lambda self, slot=f'_{field}': getattr(self, slot)))
            
        # create __eq__ method
        def eq(self, other):
            if isinstance(other, cls_object):
                # ensure each corresponding field is equal
                self_fields = [getattr(self, field) for field in cls_object._fields]
                other_fields = [getattr(other, field) for field in cls_object._fields]
                return self_fields == other_fields
            return False
        setattr(cls_object, '__eq__', eq)

        # create __hash__ method
        def hash_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            return hash(tuple(field_values))
        setattr(cls_object, '__hash__', hash_)
        
        # create __str__ method
        def str_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            field_values_joined = ', '.join(map(str, field_values))  # make every value a string
            return f'{cls_object.__name__}({field_values_joined})'
        setattr(cls_object, '__str__', str_)
        
        # create __repr__ method
        def repr_(self):
            field_values = (getattr(self, field) for field in cls_object._fields)
            field_key_values = (f'{key}={value}' for key, value in zip(cls_object._fields, field_values))
            field_key_values_str = ', '.join(field_key_values)
            return f'{cls_object.__name__}({field_key_values_str})'
        setattr(cls_object, '__repr__', repr_)
        
        return cls_object

In [16]:
class Person(metaclass=SlottedStruct):
    _fields = ['name', 'age']

    def __init__(self, name, age):
        self._name = name
        self._age = age

vars(Person)

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 1,
              '_fields': ['name', 'age'],
              '__init__': <function __main__.Person.__init__(self, name, age)>,
              '__static_attributes__': ('_age', '_name'),
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              '__slots__': ['_name', '_age'],
              'name': <property at 0x1ec5508be20>,
              'age': <property at 0x1ec55089080>,
              '__eq__': <function __main__.SlottedStruct.__new__.<locals>.eq(self, other)>,
              '__hash__': <function __main__.SlottedStruct.__new__.<locals>.hash_(self)>,
              '__str__': <function __main__.SlottedStruct.__new__.<locals>.str_(self)>,
              '__repr__': <function __main__.SlottedStruct.__new__.<locals>.repr_(self)>})

All properties, dunder methods and slots are well defined!

And now, we can use this metaclass for any of our other classes too that need to follow the same pattern: slots for all the fields, read-only properties for all the fields, and equality, hashing, repr and str as implemented.

In [17]:
class Point2D(metaclass=SlottedStruct):
    _fields = ['x', 'y']
    
    def __init__(self, x, y):
        self._x = x
        self._y = y
        
class Point3D(metaclass=SlottedStruct):
    _fields = ['x', 'y', 'z']
    
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z

In [18]:
p1 = Point2D(1, 2)
p2 = Point2D(1, 2)
p3 = Point2D(0, 0)

In [19]:
repr(p1), str(p1), hash(p1), p1.x, p1.y

('Point2D(x=1, y=2)', 'Point2D(1, 2)', -3550055125485641917, 1, 2)

# 12 - Metaprogramming - Application 2

There's another pattern we can implement using metaprogramming - Singletons.

We have seen singleton objects - objects such as `None`, `True` or `False` for example.

No matter where we create them in our code, they always refer to the **same** object.

Here's how we can create a singleton.

In [1]:
class Hundred:
    _existing_instance = None  # a class attribute!
    
    def __new__(cls):
        if not cls._existing_instance:
            new_instance = super().__new__(cls)
            setattr(new_instance, 'name', 'hundred')
            setattr(new_instance, 'value', 100)
            cls._existing_instance = new_instance
            
        return cls._existing_instance

In [2]:
h1 = Hundred()

In [3]:
h2 = Hundred()

In [4]:
h1 is h2

True

So this works, but if you need to have multiple of these singleton objects, the code will just become repetitive.

Metaclasses to the rescue!

But we're using metaclasses in a different way.

Instead of using them to control how classes are created, we're using them to control how they're *instantiated*.

Therefore, we don't want to override `__new__` because this controls the `Hundred` class creation, as opposed to instances of the `Hundred` class.

Instead, we want to override the `__call__` method, but not the one within the `Hundred` class as this makes our instances callable (`h1()`). So, we need to go one level up...

We need to override `__call__` in our metaclass. Calling `Hundred()` will trigger this method, and this is where we can implement the singleton behaviour.

In [9]:
class Singleton(type):
    def __call__(cls, *args, **kwargs):
        if getattr(cls, 'existing_instance', None) is None:  # we are using getattr approach so that we can set a default of None if not found.
            cls.existing_instance = super().__call__(*args, **kwargs)
            
        return cls.existing_instance

In [10]:
class Hundred(metaclass=Singleton):
    value = 100

In [11]:
h1 = Hundred()
h2 = Hundred()

In [12]:
h1 is h2

True

Now let's check that inheritance still works

In [16]:
class HundredFold(Hundred):
    value = 100 * 100

hf1 = HundredFold()

This looks like it worked, but did it?

In [17]:
h1 is hf1

True

Nope! It seems that the `existing_instance` attribute is being checked against the `Hundred` class instead of the `HundredFold` class. This is because when `HundredFold` inherited from `Hundred`, it also inherited the class attribute `existing_instance`.

In [18]:
vars(HundredFold)

mappingproxy({'__module__': '__main__', 'value': 10000, '__doc__': None})

In [19]:
vars(Hundred)

mappingproxy({'__module__': '__main__',
              'value': 100,
              '__dict__': <attribute '__dict__' of 'Hundred' objects>,
              '__weakref__': <attribute '__weakref__' of 'Hundred' objects>,
              '__doc__': None,
              'existing_instance': <__main__.Hundred at 0x176fd8d5c30>})

What's the solution?

We need to check if we have an `existing_instance` for each specific `type`. This means we can't just store the existing instance in the class itself. 

Instead, we need to create a dictionary in the metaclass where we correspond types (classes) with the singleton instances.

In [21]:
class Singleton(type):
    instances = {}
    
    def __call__(cls, *args, **kwargs):
        existing_instance = Singleton.instances.get(cls, None)
        if existing_instance is None:  
            Singleton.instances[cls] = super().__call__(*args, **kwargs)
            
        return Singleton.instances[cls] 

Let's test it out:

In [22]:
class Hundred(metaclass=Singleton):
    value = 100
    
class Thousand(metaclass=Singleton):
    value = 1000
    
class HundredFold(Hundred):
    value = 100 * 100

In [23]:
h1 = Hundred()
h2 = Hundred()

In [24]:
h1 is h2

True

In [25]:
t1 = Thousand()
t2 = Thousand()

In [26]:
t1 is t2

True

In [27]:
h1 is t1

False

Perfect!

# 13 - Metaprogramming - Application 3

Let's say we have some `.ini` files that hold various application configurations. We want to read those `.ini` files into an object structure so that we can access the data in our config files using dot notation.

Just some useful definitions:
- text written in square brackets are called sections, e.g. `[Database]`
- text that follows sections are called key-value pairs or items, e.g. `db_name=my_database`.

We can access the sections of `ConfigParser` using the `.sections()` method.

Let's start by creating some `.ini` files:

In [8]:
with open('s14-files/prod.ini', 'w') as prod, open('s14-files/dev.ini', 'w') as dev:
    prod.write('[Database]\n')
    prod.write('db_host=prod.mynetwork.com\n')
    prod.write('db_name=my_database\n')
    prod.write('\n[Server]\n')
    prod.write('port=8080\n')
    
    dev.write('[Database]\n')
    dev.write('db_host=dev.mynetwork.com\n')
    dev.write('db_name=my_database\n')
    dev.write('\n[Server]\n')
    dev.write('port=3000\n')

Note: I could have used the `configparser` module to write out these ini files, but we don't have to - generally these config files are created and edited manually. We will use `configparser` to load up the config files though.

When we start our program, we want to load up one of these files into a config object of some sort.

We could certainly do it this way:

In [10]:
import configparser

class Config:
    def __init__(self, env='dev'):
        print(f'Loading config from {env} file...')
        config = configparser.ConfigParser()
        file_name = f's14-files/{env}.ini'
        config.read(file_name)
        self.db_host = config['Database']['db_host']
        self.db_name = config['Database']['db_name']
        self.port = config['Server']['port']

In [11]:
config = Config('dev')

Loading config from dev file...


In [17]:
config.__dict__

{'db_host': 'dev.mynetwork.com', 'db_name': 'my_database', 'port': '3000'}

The issue with this approach is that if we want to use the `config` in multiple places, we need to create multiple instances of `Config`.

This will unnecessarily run the `ConfigParser`.

One way around is to make `Config` a singleton. Alternatively, we can instantiate it in one place e.g. `config.py` and then use that object everywhere. This is fine, but one problem we have is the lack of information we get back when using `help()` on our class.

Another issue is the manual setting of instance attributes e.g. `self.db_name = config['Database']['db_name']`. We'd prefer to do this programatically. Ideally, we'd want to have these sections as individual instances or dictionaries within our `Config` class too. 

We can implement this by creating a `Section` class which will populate its attributes based off a dictionary's key-value pairs:

In [18]:
import configparser

class Section:
    def __init__(self, name, item_dict):
        for key, value in item_dict.items():
            setattr(self, key, value)

class Config:
    def __init__(self, env='dev'):
        print(f'Loading config from {env} file...')
        config = configparser.ConfigParser()
        file_name = f's14-files/{env}.ini'
        config.read(file_name)
        for section_name in config.sections():
            section = Section(section_name, config[section_name])
            setattr(self, section_name.casefold(), section)

In [19]:
config = Config('dev')

Loading config from dev file...


In [20]:
vars(config)

{'database': <__main__.Section at 0x13ff6ac23b0>,
 'server': <__main__.Section at 0x13ff6a5f760>}

In [21]:
vars(config.database)

{'db_host': 'dev.mynetwork.com', 'db_name': 'my_database'}

We've solved most issues here except for the lack of useful information from `help(Config)`/`help(config)`. This is because we are creating instance attributes instead of class attributes.

To get around this, we can use metaclasses to set up these attributes as class attributes, not instance attributes.

To keep things a little simpler, we're going to create two distinct metaclasses. One for the sections in the config file, and one that combines the sections together - very similar to what we did with our original `Config` class.

One key difference, is that each `Section` class instance, will be a brand new class, created via its metaclass.

Let's write the `Section` metaclass first.

In [24]:
class SectionType(type):
    def __new__(mcls, name, bases, cls_dict, section_name, items_dict):
        cls_dict['__doc__'] = f'Configs for {section_name} section'
        cls_dict['section_name'] = section_name
        for key, value in items_dict.items():
            cls_dict[key] = value
        return super().__new__(mcls, name, bases, cls_dict)

We had to add `section_name` and `items_dict` to our arguments because we will use these to update the class' namespace (`cls_dict`) during initialisation of the class.

We can now create `Section` classes for different sections in our configs, passing the metaclass the section name, and a dictionary of the values it should create as class attributes.

In [19]:
class DatabaseSection(metaclass=SectionType, section_name='database', items_dict={'db_name': 'my_database', 'host': 'myhost.com'}):
    pass

Again, we want to create these sections programatically instead of passing in the `section_name` and `items_dict` straight into the class constructor.

Therefore, we *can't* use the declarative approach for creating classes i.e. `class DatabaseSection(...)`.

Instead, we need to write it imperatively. Here's a quick reminder. The following two approaches are identical:
```python
class MyClass:
    pass

MyClass = type('MyClass', (object,), {})  # name, bases, cls_dict/namespace
```

This is the approach we'll take to programmatically creating our different sections inside our `Config` metaclass which we shall call `ConfigType`:

In [27]:
class ConfigType(type):
    def __new__(cls, name, bases, cls_dict, env):
        """
        env : str
            The environment we are loading the config for (e.g. dev, prod)
        """
        cls_dict['__doc__'] = f'Configurations for {env}.'  # these are the class attrs of `Config` created below
        cls_dict['env'] = env  # these are the class attrs of `Config` created below
        config = configparser.ConfigParser()
        file_name = f's14-files/{env}.ini'
        config.read(file_name)
        for section_name in config.sections():
            class_name = section_name.capitalize()
            class_attribute_name = section_name.casefold()
            section_items = config[section_name]
            bases = (object, )
            section_cls_dict = {}
            # create a new Section class for this section
            Section = SectionType(
                class_name, 
                bases, 
                section_cls_dict, 
                section_name=section_name, 
                items_dict=section_items
            )
            # And assign it to an attribute in the main config class
            cls_dict[class_attribute_name] = Section
            
        return super().__new__(cls, name, bases, cls_dict)

Now we can create config classes for each of our environments:

In [28]:
class Config(metaclass=ConfigType, env='dev'):
    pass

This has autocompletion and `help` fully functional: 

In [29]:
help(Config)

Help on class Config in module __main__:

class Config(builtins.object)
 |  Configurations for dev.
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  database = <class '__main__.Database'>
 |      Configs for Database section
 |  
 |  
 |  env = 'dev'
 |  
 |  server = <class '__main__.Server'>
 |      Configs for Server section



In [30]:
help(Config.database)

Help on class Database in module __main__:

class Database(builtins.object)
 |  Configs for Database section
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  db_host = 'dev.mynetwork.com'
 |  
 |  db_name = 'my_database'
 |  
 |  section_name = 'Database'



In [31]:
Config.database.db_name

'my_database'

If we place these `Config` class in a module and import it somewhere, all of these steps will run.

But, if we import it somewhere else, it won't run again because the module is already in our `sys.modules` namespace. So, we got around making a singleton!

# 14 - Attribute Read Accessors

#### Lecture

When we do something like `person.name`, Python has to look up that attribute's value. 

But this can be living in a number of places:
- instance dictionary
- descriptor (attribute is class attribute with `__get__` implemented)
- plain class attribute
- in a parent class

So how does Python figure out which one to call?

**The `__getattribute__` Method**

This is the method that will be called when we use the dot notation.

It has a very complex default implementation, but it can be overridden -> `__getattribute__(self, name)`

Due to the complexity, we often delegate back to `super().__getattribute__` to do the heavy lifting.

But you may have noticed that `super().__getattribute__` is also performing an attribute lookup due to the dot notation.

In fact, **dunder methods are looked up differently than our own attributes**.

**The `__getattr__` Method**

If `__getattribute__` method cannot find the requested attribute, it raises an `AttributeError`.

Python catches this and then tries to call `__getattr__`. Think of it as a **last resort**.

Now, the default implementation of this method is just to re-raise the `AttributeError`. In other words, it does nothing.

But, it's there to be overridden as it's generally easier to override this method than `__getattribute__`.

Here's the entire flow for an attribute lookup:

<img src=s14-images/14.1.png width=900 />

This makes it more clear why we override `__getattr__` instead of `__getattribute__`.

**Overriding access for class attributes**

What we've discussed so far has been overriding instance attributes since `__getattribute__` and `__getattr__` are instance methods.

How do we override **class attribute** access?

We need to override `__getattribute__` and `__getattr__` in the metaclass. 

We'll see this in the latter part of the coding section.

#### Coding

When overriding `__getattr__`, we must be careful not to cause infinite recursion.

This will typically occur if we attempt to access an attribute within `__getattr__` that does not exist, which will call `__getattr__` over and over again...

The solution is to use a `try-except` with the attribute lookup mechanism in the `super()` object: `super().__getattribute__` in the `try`, catching the `AttributeError` in the `except`. If we succeed in finding the attribute, all is good. If we don't, then we've caught the exception and we can re-raise the `AttributeError` to gracefully exit.

This is a common pattern when overriding accessors.

In [4]:
class Person:
    def __getattr__(self, name):
        alt_name = '_' + name
        print(f'Could not find {name}, trying {alt_name}...')
        try:
            return super().__getattribute__(alt_name)
        except AttributeError:
            raise AttributeError(f'Could not find {name} or {alt_name}')  # in Python's default implementation, `__getattr__` would be
                                                                          # called here. See above attribute lookup flow.

In [5]:
p = Person()

In [6]:
try:
    p.age
except AttributeError as ex:
    print(type(ex).__name__, ex)

Could not find age, trying _age...
AttributeError Could not find age or _age


##### Example 1

Here we're going to create a class that behaves a little bit like `defaultdict`. 

If an attribute is requested that does not exist, we're going to set in in the instance, to some default value, and then return it.

In [7]:
class DefaultClass:
    def __init__(self, attribute_default=None):
        self._attribute_default = attribute_default
        
    def __getattr__(self, name):
        print(f'{name} not found. creating it and setting it to default...')
        setattr(self, name, self._attribute_default)
        return self._attribute_default

In [8]:
d = DefaultClass('NotAvailable')

In [9]:
d.test

test not found. creating it and setting it to default...


'NotAvailable'

In [10]:
d.__dict__

{'_attribute_default': 'NotAvailable', 'test': 'NotAvailable'}

And of course, the next time we request it, the `__getattr__` is no longer called:

In [11]:
d.test

'NotAvailable'

Now that we have this class defined, we could also inherit from it to provide this functionality to other classes:

In [22]:
class Person(DefaultClass):
    def __init__(self, name):
        super().__init__('Unavailable')
        self.name = name

In [23]:
p = Person('Raymond')

In [24]:
p.name

'Raymond'

In [25]:
p.age

age not found. creating it and setting it to default...


'Unavailable'

##### Example 3: Overriding `__getattribute__`

(Example 2 in full notes)

As we discussed in the lecture, `__getattribute__` is called for **every** attribute access on our object.

We'll come back to more examples of this, but let's do a simple example, where we want to disallow accessing any attribute names that start with an underscore:

In [12]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    def __getattribute__(self, name):
        if name.startswith('_') and not name.startswith('__'):
            raise AttributeError(f'Forbidden access to {name}')
        return super().__getattribute__(name)

In [13]:
p = Person('Eric', 78)

In [14]:
p.__dict__

{'_name': 'Eric', '_age': 78}

Now let's implement properties for `name` and `age`:

In [15]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    def __getattribute__(self, name):
        if name.startswith('_') and not name.startswith('__'):
            raise AttributeError(f'Forbidden access to {name}')
        return super().__getattribute__(name)
    
    @property
    def name(self):
        return self._name
    
    @property
    def age(self):
        return self._age

I hope before we even run this, that you realize we are going to have an issue...

In the properties, what did we do? We accessed `self._name` and `self._age`.

How is Python going to look up those attributes? By using the `__getattribute__` method - and we just stopped access to variables that start with a single underscore!

In [16]:
p = Person('Python', 42)

In [17]:
try:
    p.name
except AttributeError as ex:
    print(ex)

Forbidden access to _name


Somehow we need to bypass our custom implementation of `__getattribute__`. And we do that by delegating the attribute lookup to `super()` - that will use the standard lookup method (define in `object` in this case), and not our custom method.

In [18]:
class Person:
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    def __getattribute__(self, name):
        if name.startswith('_') and not name.startswith('__'):
            raise AttributeError(f'Forbidden access to {name}')
        return super().__getattribute__(name)
    
    @property
    def name(self):
        return super().__getattribute__('_name')
    
    @property
    def age(self):
        return super().__getattribute__('_age')

In [19]:
p = Person('Python', 42)

In [20]:
p.name

'Python'

The main takeaway from this section is that, when you really want the attribute without being blocked by your own overridden implementations of the accessors, **use `super().__getattribute__`.**

##### Overriding Class Attributes

To override instance attributes, we override `__getattribute__` in the class. 

It therefore follows that, to override class attributes, we must override `__getattribute__` in the **metaclass**.

Here's a simple example.

In [21]:
class MetaLogger(type):
    def __getattribute__(self, name):  # called on EVERY class attribute.
        print('class __getattribute__ called...')
        return super().__getattribute__(name)
    
    def __getattr__(self, name):  # called when class attribute doesn't exist.
        print('class __getattr__ called...')
        return 'Not Found'

In [52]:
class Account(metaclass=MetaLogger):
    apr = 10

In [53]:
Account.apr

class __getattribute__ called...


10

In [54]:
Account.apy

class __getattribute__ called...
class __getattr__ called...


'Not Found'

Apart from the fact that we defined these methods in the metaclass, everything else works the same way.

##### Gets called for Method access as well

When we call our custom methods in a custom class, the method needs to be retrieved from the instance as well - so it uses the `__getattribute__` and `__getattr__` methods as well.

In [55]:
class MyClass:
    def __getattribute__(self, name):
        print(f'__getattribute__ called... for {name}')
        return super().__getattribute__(name)
    
    def __getattr__(self, name):
        print(f'__getattr__ called... for {name}')
        raise AttributeError(f'{name} not found')
    
    def say_hello(self):
        return 'hello'

In [56]:
m = MyClass()

In [57]:
m.say_hello()

__getattribute__ called... for say_hello


'hello'

In [58]:
m.other()

__getattribute__ called... for other
__getattr__ called... for other


AttributeError: other not found

# 15 - Attribute Write Accessors

The flow diagram for attribute setters is a lot more simple.

<img src=s14-images/14.2.png width=900 />

The same caveats regarding infinite recursion apply here as well, therefore we'll often need to use `super().__setattr__` as a guard.

For class attributes, we override in the metaclass similar to how we did for `__getattr__`.

Here's the basic pattern:

In [1]:
class Person:
    def __setattr__(self, name, value):
        print('setting instance attribute...')
        super().__setattr__(name, value)

In [2]:
p = Person()

In [3]:
p.name = 'Guido'

setting instance attribute...


In order to override this setter for class attributes we would have to define it in the metaclass:

In [1]:
class MyMeta(type):
    def __setattr__(self, name, value):
        print('setting class attribute...')
        return super().__setattr__(name, value)
    
class Person(metaclass=MyMeta):
    def __setattr__(self, name, value):
        print('setting instance attribute...')
        super().__setattr__(name, value)

In [2]:
Person.test = 'test'

setting class attribute...


In [3]:
p = Person()
p.test = 'test'

setting instance attribute...


And as we discussed in the lecture, if our `__setattr__` is setting a **data** descriptor, then it calls the descriptor's `__set__` method instead, after first calling `__setattr__`:

In [5]:
class MyNonDataDesc:
    def __get__(self, instance, owner_class):
        print('__get__ called on non-data descriptor...')
        
class MyDataDesc:
    def __set__(self, instance, value):
        print('__set__ called on data descriptor...')
        
    def __get__(self, instance, owner_class):
        print('__get__ called on data descriptor...')

In [6]:
class MyClass:
    non_data_desc = MyNonDataDesc()
    data_desc = MyDataDesc()
    
    def __setattr__(self, name, value):
        print('__setattr__ called...')
        super().__setattr__(name, value)

In [7]:
m = MyClass()

In [8]:
m.__dict__

{}

In [9]:
m.data_desc = 100

__setattr__ called...
__set__ called on data descriptor...


So what happened here was `__setattr__` was called, as is default. 

Then, when the default implementation was called (through `super().__setattr__(name, value)`), it figured out that `name` was a **data** descriptor due to it having implemented `__set__`; therefore, it called `__set__`.

Now let's call the **non-data** descriptor.

In [10]:
m.non_data_desc = 200

__setattr__ called...


As expected, since Python determined that `name` was *not* a **data** descriptor, it **did not** call `__set__`.

Looking back at our flow diagram for the setter, we expect it to populate/update our `__dict__`.

In [11]:
m.__dict__

{'non_data_desc': 200}

# 16 - Accessors - Application

Another useful application of `__getattr__` and `__setattr__` is dealing with objects where we may not know the attributes in advance.

Consider this scenario where we have a database with various tables and fields. We want to create a class that allows us to retrieve data from these tables.

We could certainly write a class for each specific table, and hardcode the fields as properties in the class - but that's going to create repetitive code, and anytime there is a new table or the schema of an existing table changes we'll have to revise our code.

I'm going to simulate a database here by using dictionaries. The outer dictionary will contain tables (as keys), and each table will contain records with a numeric key for each record.

In [12]:
DB = {
    'Person': {
        1: {'first_name': 'Isaac', 'last_name': 'Newton', 'born': 1642, 'country_id': 1},
        2: {'first_name': 'Gottfried', 'last_name': 'von Leibniz', 'born': 1646, 'country_id': 5},
        3: {'first_name': 'Joseph', 'last_name': 'Fourier', 'born': 1768, 'country_id': 3},
        4: {'first_name': 'Bernhard', 'last_name': 'Riemann', 'born': 1826, 'country_id': 5},
        5: {'first_name': 'David', 'last_name': 'Hilbert', 'born': 1862 , 'country_id': 5},
        6: {'first_name': 'Srinivasa', 'last_name': 'Ramanujan', 'born': 1887, 'country_id': 4},
        7: {'first_name': 'John', 'last_name': 'von Neumann', 'born': 1903, 'country_id': 2},
        8: {'first_name': 'Andrew', 'last_name': 'Wiles', 'born': 1928, 'country_id': 6}
    },
    'Country': {
        1: {'name': 'United Kingdom', 'capital': 'London', 'continent': 'Europe'},
        2 :{'name': 'Hungary', 'capital': 'Budapest', 'continent': 'Europe'},
        3: {'name': 'France', 'capital': 'Paris', 'continent': 'Europe'},
        4: {'name': 'India', 'capital': 'New Delhi', 'continent': 'Asia'},
        5: {'name': 'Germany', 'capital': 'Berlin', 'continent': 'Europe'},
        6: {'name': 'USA', 'capital': 'Washington DC', 'continent': 'North America'}
        }
}

Now we could certainly do something like this for each table:

In [13]:
class Country:
    def __init__(self, id_):
        if _id in DB['Country']:
            self._db_record = DB['Country'][id_]
        else:
            raise ValueError(f'Record not found (Country.id={id_})')

    @property
    def name(self):
        return self._db_record['name']
    
    @property
    def capital(self):
        return self._db_record['capital']
    
    @property
    def continent(self):
        return self._db_record['continent']

And we would have to do the same thing with the `Person` table, and any other table we want from our database. Tedious and repetitive code!!

We could create a metaclass that inspects the table structure and creates the appropriate fields, that would work well with code completion for example. 

But if we don't want to get too fancy, we can instead just use `__getattr__`. We'll implement the `__setattr__` as well, but of course in a real database situation you would need to implement some mechanism to persist the changes back to the database.

We are going to create a `DBTable` class that will be used to represent a table in the database, and we'll make it callable so we can pass the record id to the instance, which will return a `DBRecord` object that we can then use to access the fields in the table.

Let's write the `DBRecord` class first. This class will be passed a database record (so a dictionary in this example), and will be tasked with looking up "fields" (keys in this example) in the table (dictionary).

In [14]:
class DBRecord:
    def __init__(self, db_record_dict):
        # again, careful how you set a property on instances of this class
        # because we are overriding __setattr__ we cannot just use 
        # self._record = db_record_dict
        # this will call OUR version of `__setattr__`, which attempts to 
        # see if name is in _record - but _record does not exist yet, so it will
        # call __getattr__, which in turn tries to check if that is contained in _record
        # so, infinite recursion.
        # What we want to here is BYPASS our custom __setattr__ - so we'll use
        # the one in the superclass.
        super().__setattr__('_record', db_record_dict)    
        
    def __getattr__(self, name):
        # here we could write
        #     if name in self._record 
        # since this method should not get called
        # before _record as been created.
        # But just to be on the safe side, I'm still going to use super
        if name in super().__getattribute__('_record'):
            # this is totally safe because we have just ensured that
            # `_record` exists so we can't get infinite recursion.
            return self._record[name]  
        else:
            raise AttributeError(f'Field name {name} does not exist.')

    def __setattr__(self, name, value):
        # and again here, we could write
        # if name in self._record, but I'm still going to use super
        if name in super().__getattribute__('_record'):
            # super().__setattr__(name, value)
            self._record[name] = value
        else:
            raise AttributeError(f'Field name {name} does not exist.')

Next, we define the `DBTable` class. It will be initialized with the name of the table we want to use in our instance. Furthermore we'll make it callable (passing in the record id) and that shoudl return an instance of `DBRecord` for the particular record.

In [15]:
class DBTable:
    def __init__(self, db, table_name):
        if table_name not in db:
            raise ValueError(f'The table {table_name} does not exist in the database.')
        self._table_name = table_name
        self._table = db[table_name]
        
    @property
    def table_name(self):
        return self._table_name
    
    def __call__(self, record_id):
        if record_id not in self._table:
            raise ValueError(f'Specified id ({record_id}) does not exist '
                             f'in table {self._table_name}')
        return DBRecord(self._table[record_id])

And now we can use our classes this way:

In [16]:
tbl_person = DBTable(DB, 'Person')
tbl_country = DBTable(DB, 'Country')

In [18]:
person_1 = tbl_person(1)

In [19]:
person_1.first_name, person_1.last_name, person_1.born, person_1.country_id

('Isaac', 'Newton', 1642, 1)

We can simulate a JOIN by doing the following, for example:

In [21]:
country_1 = tbl_country(person_1.country_id)

In [22]:
country_1.name, country_1.capital

('United Kingdom', 'London')

END OF COURSE!!!